# Building RESNET Residual Networks

In [1]:
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model, load_model



2023-01-23 12:49:42.569512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 12:49:43.362169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-23 12:49:43.362202: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-23 12:49:45.312308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

<a name='2'></a>
## 1 - The Problem of Very Deep Neural Networks

Last week, you built your first convolutional neural networks: first manually with numpy, then using Tensorflow and Keras. 

In recent years, neural networks have become much deeper, with state-of-the-art networks evolving from having just a few layers (e.g., AlexNet) to over a hundred layers.

* The main benefit of a very deep network is that it can represent very complex functions. It can also learn features at many different levels of abstraction, from edges (at the shallower layers, closer to the input) to very complex features (at the deeper layers, closer to the output). 

* However, using a deeper network doesn't always help. A huge barrier to training them is vanishing gradients: very deep networks often have a gradient signal that goes to zero quickly, thus making gradient descent prohibitively slow.

* More specifically, during gradient descent, as you backpropagate from the final layer back to the first layer, you are multiplying by the weight matrix on each step, and thus the gradient can decrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode," from gaining very large values). 

* During training, you might therefore see the magnitude (or norm) of the gradient for the shallower layers decrease to zero very rapidly as training proceeds, as shown below: 

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> <b>Figure 1</b> </u><font color='purple'>  : <b>Vanishing gradient</b> <br> The speed of learning decreases very rapidly for the shallower layers as the network trains </center></caption>

We can solve this problem by building a Residual Network!

<a name='3'></a>
## 2 - Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers:  

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> <b>Figure 2</b> </u><font color='purple'>  : A ResNet block showing a skip-connection <br> </center></caption>

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are the same or different. We will implement both of them: the "identity block" and the "convolutional block."

<a name='3-1'></a>
### 2.1 - The Identity Block

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say $a^{[l]}$) has the same dimension as the output activation (say $a^{[l+2]}$). To flesh out the different steps of what happens in a ResNet's identity block, here is an alternative diagram showing the individual steps:

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> <b>Figure 3</b> </u><font color='purple'>  : <b>Identity block.</b> Skip connection "skips over" 2 layers. </center></caption>
    

We will implement a slightly more powerful version of this identity block, in which the skip connection "skips over" 3 hidden layers rather than 2 layers. It looks like this: 

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
    <caption><center> <u> <font color='purple'> <b>Figure 4</b> </u><font color='purple'>  : <b>Identity block.</b> Skip connection "skips over" 3 layers.</center></caption>

In [2]:
def identity_block(x, f, filters, training=True):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
   
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    x_shortcut=x
    
    F1,F2,F3=filters
    #1st component of main path
    x=Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding="valid")(x)
    x=BatchNormalization(axis=3)(x,training=training) #default axis
    x=Activation("relu")(x)
    
    #2nd component of main path
    x=Conv2D(filters=F2,kernel_size=f,strides=(1,1),padding="same")(x)
    x=BatchNormalization(axis=3)(x,training=training) #default axis
    x=Activation("relu")(x)
    
    #3rd component of main path
    x=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding="valid")(x)
    x=BatchNormalization(axis=3)(x,training=training) #default axis
    
    #Add x_shortcut to main path and pass it to activation layer
    x=Add()([x_shortcut,x])
    x=Activation("relu")(x)
    
    return x
    

<a name='3-2'></a>
### 2.2 - The Convolutional Block

The ResNet "convolutional block" is the second block type. You can use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path: 

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> <b>Figure 4</b> </u><font color='purple'>  : <b>Convolutional block</b> </center></caption>

* The CONV2D layer in the shortcut path is used to resize the input $x$ to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. 
* For example, to reduce the activation dimensions's height and width by a factor of 2, you can use a 1x1 convolution with a stride of 2. 
* The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step. 


In [3]:

def convolutional_block(x, f, filters, s = 2, training=True):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
   
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    x_shortcut=x
    
    F1,F2,F3=filters
    
    #1st block of main path
    x=Conv2D(filters=F1,kernel_size=(1,1),strides=(s,s),padding='valid')(x)
    x=BatchNormalization(axis=3)(x,training=training)
    x=Activation('relu')(x)
    
    #2nd block of main path
    x=Conv2D(filters=F2,kernel_size=f,strides=(1,1),padding='same')(x)
    x=BatchNormalization(axis=3)(x,training=training)
    x=Activation('relu')(x)
    
    #3rd block of main path
    x=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid')(x)
    x=BatchNormalization(axis=3)(x,training=training)
    
    #shortcut path
    x_shortcut=Conv2D(filters=F3,kernel_size=(1,1),strides=(s,s),padding='valid')(x_shortcut)
    x_shortcut=BatchNormalization(axis=3)(x_shortcut,training=training)
    
    #Add x_shortcut to main path
    x=Add()([x,x_shortcut])
    x=Activation('relu')(x)
    
    return x

<a name='4'></a>  
## 4 - Building ResNet Model (50 layers)

The following figure describes in detail the architecture of this neural network. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means you should stack 3 identity blocks together.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> <b>Figure 5</b> </u><font color='purple'>  : <b>ResNet-50 model</b> </center></caption>

    
The details of this ResNet-50 model are:
- Zero-padding pads the input with a pad of (3,3)
- Stage 1:
    - The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). 
    - BatchNorm is applied to the 'channels' axis of the input.
    - MaxPooling uses a (3,3) window and a (2,2) stride.
- Stage 2:
    - The convolutional block uses three sets of filters of size [64,64,256], "f" is 3, and "s" is 1.
    - The 2 identity blocks use three sets of filters of size [64,64,256], and "f" is 3.
- Stage 3:
    - The convolutional block uses three sets of filters of size [128,128,512], "f" is 3 and "s" is 2.
    - The 3 identity blocks use three sets of filters of size [128,128,512] and "f" is 3.
- Stage 4:
    - The convolutional block uses three sets of filters of size [256, 256, 1024], "f" is 3 and "s" is 2.
    - The 5 identity blocks use three sets of filters of size [256, 256, 1024] and "f" is 3.
- Stage 5:
    - The convolutional block uses three sets of filters of size [512, 512, 2048], "f" is 3 and "s" is 2.
    - The 2 identity blocks use three sets of filters of size [512, 512, 2048] and "f" is 3.
- The 2D Average Pooling uses a window of shape (2,2).
- The 'flatten' layer doesn't have any hyperparameters.
- The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation.

    

In [4]:

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    x_input=Input(shape=input_shape)
    
    x=ZeroPadding2D((3,3))(x_input)
    
    #stage 1
    x=Conv2D(64,(7,7),strides=(2,2))(x)
    x=BatchNormalization(axis=3)(x)
    x=Activation('relu')(x)
    x=MaxPooling2D((3,3),strides=(2,2))(x)
    
    #stage 2
    x=convolutional_block(x,f=3,filters=[64,64,256],s=1)
    x=identity_block(x,f=3,filters=[64,64,256])
    x=identity_block(x,f=3,filters=[64,64,256])
    
    #stage 3
    x=convolutional_block(x,f=3,filters=[128,128,512],s=2)
    x=identity_block(x,f=3,filters=[128,128,512])
    x=identity_block(x,f=3,filters=[128,128,512])
    x=identity_block(x,f=3,filters=[128,128,512])
    
    #stage 4
    x=convolutional_block(x,f=3,filters=[256,256,1024],s=2)
    x=identity_block(x,f=3,filters=[256,256,1024])
    x=identity_block(x,f=3,filters=[256,256,1024])
    x=identity_block(x,f=3,filters=[256,256,1024])
    x=identity_block(x,f=3,filters=[256,256,1024])
    x=identity_block(x,f=3,filters=[256,256,1024])
    
    #stage 5
    x=convolutional_block(x,f=3,filters=[512,512,2048],s=2)
    x=identity_block(x,f=3,filters=[512,512,2048])
    x=identity_block(x,f=3,filters=[512,512,2048])
    
    #Avg Pool
    x=AveragePooling2D((2,2))(x)
    
    #output
    x=Flatten()(x)
    x=Dense(classes,activation='softmax')(x)
    
    model=Model(inputs=x_input,outputs=x)
    
    return model

In [5]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

2023-01-23 12:49:47.700551: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-23 12:49:47.700636: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-23 12:49:47.700686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2413d2c4aac4): /proc/driver/nvidia/version does not exist
2023-01-23 12:49:47.701598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                   

In [6]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])